In [1]:
from azureml.core import Workspace, Run
ws = Workspace.from_config()
print(ws)

Workspace.create(name='huytung-ws', subscription_id='1f5e877f-48a6-4973-aa3b-ceead3340ce7', resource_group='huytung-rsgroup')


In [2]:
datastore = ws.get_default_datastore()

In [3]:
datastore.upload(src_dir='../data',
                 target_path='datasets/diabetes',
                 overwrite=True)

Uploading an estimated of 1 files
Uploading ../data/diabetes.csv
Uploaded ../data/diabetes.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_3dfca975d41b4d6f9c6b0eb36c2460ad

In [4]:
datastore

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-553468aa-0d06-41b5-9905-8c4c963277fd",
  "account_name": "huytungws8293462828",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [5]:
datastore_paths = [(datastore, 'datasets/diabetes/diabetes.csv')]
datastore_paths

[({
    "name": "workspaceblobstore",
    "container_name": "azureml-blobstore-553468aa-0d06-41b5-9905-8c4c963277fd",
    "account_name": "huytungws8293462828",
    "protocol": "https",
    "endpoint": "core.windows.net"
  },
  'datasets/diabetes/diabetes.csv')]

In [6]:
from azureml.core import Dataset
dataset = Dataset.Tabular.from_delimited_files(path=datastore_paths)

In [7]:
from azureml.core import Environment
env = Environment.from_existing_conda_environment(name="huytung_env",
                                                    conda_environment_name="tutorial")                                             

Exporting conda specifications for existing conda environment: tutorial


In [9]:
from azureml.core import ScriptRunConfig, Experiment
from azureml.widgets import RunDetails
# Create a script config
training_folder = './'
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                environment=env,
                                compute_target = 'huytungcomputename',
                                arguments=['--data_path', dataset.as_named_input('diabetes')]) 

# submit the experiment run
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
aml_url = run.get_portal_url()
# Show the running experiment run in the notebook widget
# RunDetails(run).show()

# Block until the experiment run has completed
run.wait_for_completion()
print(aml_url)

https://ml.azure.com/runs/mslearn-train-diabetes_1619418634_5485b9f0?wsid=/subscriptions/1f5e877f-48a6-4973-aa3b-ceead3340ce7/resourcegroups/huytung-rsgroup/workspaces/huytung-ws&tid=1fd983f3-dc44-42f8-ad66-7972e9d94659


In [10]:
# Get logged metrics and files
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Regularization Rate 0.01
Accuracy 0.774
AUC 0.8484437036668493


azureml-logs/55_azureml-execution-tvmps_e18f985b2a2a85421be77841684473bc3ce38caa77df5fd10c25655aeba88109_d.txt
azureml-logs/65_job_prep-tvmps_e18f985b2a2a85421be77841684473bc3ce38caa77df5fd10c25655aeba88109_d.txt
azureml-logs/70_driver_log.txt
azureml-logs/75_job_post-tvmps_e18f985b2a2a85421be77841684473bc3ce38caa77df5fd10c25655aeba88109_d.txt
azureml-logs/process_info.json
azureml-logs/process_status.json
logs/azureml/99_azureml.log
logs/azureml/dataprep/backgroundProcess.log
logs/azureml/dataprep/backgroundProcess_Telemetry.log
logs/azureml/job_prep_azureml.log
logs/azureml/job_release_azureml.log
outputs/diabetes_model.pkl


In [12]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model version: 1
	 Training context : Script
	 AUC : 0.8484437036668493
	 Accuracy : 0.774


